# Corrections & downstream analysis

This notebook describes a correction + downstream analysis pipeline for the analysis of a generic sample (tissue), taking as input a single, filtered anndata object.

### Loading required packages

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline



import anndata as ad
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
from matplotlib.colors import ListedColormap
import seaborn as sb
import scvelo as scv
import re
from rpy2.robjects import pandas2ri

import rpy2.rinterface_lib.callbacks
import rpy2
%load_ext rpy2.ipython

import time # for the sleep function
import os # to iterate over directories

In [ ]:
#extra settings
# A nice color scheme for visualizing gene expression
colors_2 = plt.cm.OrRd(np.linspace(0.05, 1, 128))
colors_3 = plt.cm.Greys_r(np.linspace(0.8,0.9,20))
colors_Comb = np.vstack([colors_3, colors_2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colors_Comb)

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80)
#print package version list
#sc.logging.print_versions()

### Variable inputs

In [ ]:
# Variables for reading in data:
tissue = 'MO' # name of the sample; change accordingly
conditionnames = ['C','ED','LD','PL','IC'] # name of the conditions in the preferred (arbitrary) order of display; change accordingly
# note that the paths for reading in data will be:
    # 10x matrix:    cellranger_outputs/'+ tissue +'/' + conditionnames[n] + '-' + tissue + '_primirs' + '/outs'
    # HTO data:      hto_classification/'+ tissue +'/' + conditionnames[file]+'-' + tissue + '_HTO_info.txt'
conditionlength = len(conditionnames)
filtered_file = 'outputs/' + tissue + '/terva_'+ tissue + '_qc.h5ad' # path for the input anndata object

In [ ]:
# Variables for data storage
figdirectory = 'outputs/' + tissue + '/figures/' #directory for figure saving
clusters_file = 'outputs/' + tissue + '/terva_'+ tissue + '_clusters.h5ad' # path for the output anndata object

# 1. Read in data 

In [ ]:
adata=sc.read(filtered_file,cache=True) 

In [ ]:
adata.obs.head()

# 2. Corrections

## 2.1. Normalization

In [ ]:
#Perform a clustering for scran normalization in clusters
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e4,copy=True)
sc.pp.log1p(adata_pp) 
sc.pp.pca(adata_pp, n_comps=15)
sc.pp.neighbors(adata_pp)

In [ ]:
# Leiden clustering for normalization
sc.tl.leiden(adata_pp, key_added='groups', resolution=0.3)
# for Louvain: sc.tl.louvain(adata_pp, key_added='groups', resolution=0.3)

In [ ]:
#Preprocess variables for scran normalization
input_groups = adata_pp.obs['groups']
data_mat = adata.X.T

In [ ]:
%%R
# Load all the R libraries we will be using in the notebook
library(scran)

In [ ]:
import anndata2ri
anndata2ri.activate()
from rpy2.robjects import r
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

In [ ]:
%%R -i data_mat -i input_groups -o size_factors
#normalized expression values (measured counts/size factors)
size_factors = computeSumFactors(data_mat, clusters=input_groups,min.mean=0.1)

In [ ]:
#Delete adata_pp
del adata_pp

We plot the size factors to show they are non-negative and related to the number of counts and genes per cell as expected.

In [ ]:
# Add to adata and visualize the estimated size factors
adata.obs['size_factors'] = size_factors
sc.pl.scatter(adata, 'size_factors', 'n_counts')
sc.pl.scatter(adata, 'size_factors', 'n_genes')
sb.distplot(size_factors, bins=50, kde=False)
plt.show()

Before normalizing the data, we ensure that a copy of the raw count data is kept in a separate AnnData object. This allows us to use methods downstream that require this data as input.

In [ ]:
#Keep the count data in a counts layer
adata.layers["counts"] = adata.X.copy()

In [ ]:
#Normalize adata
adata.X /= adata.obs['size_factors'].values[:,None]
sc.pp.log1p(adata)


The count data has been normalized and log-transformed with an offset of 1. The latter is performed to normalize the data distributions. The offset of 1 ensures that zero counts map to zeros. We keep this data in the '.raw' part of the AnnData object as it will be used to visualize gene expression and perform statistical tests such as computing marker genes for clusters.

In [ ]:
# Store the full data set in 'raw' as log-normalised data for statistical testing
adata.raw = adata # it does this again, check to remove from the beginning

In [ ]:
# Convert back to sparse matrix: https://github.com/theislab/scanpy/issues/456
adata.X = sp.sparse.csr_matrix(adata.X) 

## 2.2. Batch Correction

Batch correction was not performed with ComBat but it may be an optional step.
Note that ComBat batch correction requires a dense matrix format as input (which is already the case in this example).

In [ ]:
# ComBat batch correction
#sc.pp.combat(adata, key='ConditionName')

# 3. Analysis

## 3.1. HVG

We extract highly variable genes (HVGs) to further reduce the dimensionality of the dataset and include only the most informative genes. Genes that vary substantially across the dataset are informative of the underlying biological variation in the data. As we only want to capture biological variation in these genes, we select highly variable genes after normalization and batch correction. HVGs are used for clustering, trajectory inference, and dimensionality reduction/visualization, while the full data set is used for computing marker genes, differential testing, cell cycle scoring, and visualizing expression values on the data.

Here we use a standard technique for the extraction of highly variable genes from the 10X genomics preprocessing software CellRanger. Typically between 1000 and 5000 genes are selected. Here, we extract the top 4000 most variable genes for further processing. If particular genes of importance are known, one could assess how many highly variable genes are necessary to include all, or the majority, of these.

In [ ]:
#sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, flavor= "cell_ranger")
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', n_top_genes=4000) #from Theis Lab
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))
sc.pl.highly_variable_genes(adata)

The plots show how the data was normalized to select highly variable genes irrespective of the mean expression of the genes. This is achieved by using the index of dispersion which divides by mean expression, and subsequently binning the data by mean expression and selecting the most variable genes within each bin.

Highly variable gene information is stored automatically in the adata.var['highly_variable'] field. The dataset now contains:

a 'counts' layer with count data
log-normalized data in adata.raw
batch corrected data in adata.X
highly variable gene annotations in adata.var['highly_variable']
The HVG labels will be used to subselect genes for clustering and trajectory analysis.

In [ ]:
adata.var['highly_variable'].value_counts()
sum(adata.var['highly_variable'])
#adata = adata[:, adata.var['highly_variable']]
#we are not going to subset highly variable genes, since they will drive the analysis anyway

In [ ]:
adata.obs["HTO_classification"].value_counts()

## 3.2. Visualization

Visualizing scRNA-seq data is the process of projecting a high-dimensional matrix of cells and genes into a few coordinates such that every cell is meaningfully represented in a two-dimensional graph. However, the visualization of scRNA-seq data is an active area of research and each method defines 'meaningful' in its own way. Thus, it is a good idea to look at several visualizations and decide which visualization best represents the aspect of the data that is being investigated.

Overall t-SNE visualizations have been very popular in the community, however the recent UMAP algorithm has been shown to better represent the topology of the data.

Note that we do not scale the genes to have zero mean and unit variance. A lack of rescaling is equivalent to giving genes with a higher mean expression a higher weight in dimensionality reduction (despite correcting for mean offsets in PCA, due to the mean-variance relationship). We argue that this weighting based on mean expression being a biologically relevant signal. However, rescaling HVG expression is also common, and the number of publications that use this approach suggests that scaling is at least not detrimental to downstream scRNA-seq analysis.

In [ ]:
sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack') 

In [ ]:
sc.pp.neighbors(adata)

In [ ]:
sc.tl.tsne(adata, n_jobs=12) #Note n_jobs works for MulticoreTSNE, but not regular implementation)

In [ ]:
sc.tl.umap(adata,random_state=5)

In [ ]:
sc.tl.diffmap(adata)

In [ ]:
sc.pl.pca_scatter(adata, color='n_counts',save= figdirectory + tissue +'_pca_ncounts.png')
sc.pl.tsne(adata, color='n_counts',save= figdirectory + tissue +'_tsne_ncounts.png')
sc.pl.umap(adata, color='n_counts',save= figdirectory + tissue +'_umap_ncounts.png')
sc.pl.diffmap(adata, color='n_counts', components=['1,2','1,3'],save= figdirectory + tissue +'_diffmap_ncounts.png')

In [ ]:
sc.pl.pca_scatter(adata, color=["ConditionName",'n_genes','percent_mito'],save= figdirectory + tissue +'_pca_condition.png')
sc.pl.tsne(adata, color=["ConditionName",'n_genes','percent_mito'],save= figdirectory + tissue +'_tsne_condition.png')
sc.pl.umap(adata, color=["ConditionName",'n_genes','percent_mito'],save= figdirectory + tissue +'_umap_condition.png')

sc.pl.pca_scatter(adata, color=["ConditionName","HTO_classification_global","HTO_classification",'n_genes','percent_mito'],save= figdirectory + tissue +'_pca_hto.png')
sc.pl.tsne(adata, color=["ConditionName","HTO_classification_global","HTO_classification",'n_genes','percent_mito'],save= figdirectory + tissue +'_tsne_hto.png')
sc.pl.umap(adata, color=["ConditionName","HTO_classification_global","HTO_classification",'n_genes','percent_mito'],save= figdirectory + tissue +'_umap_hto.png')

PCA:

Unsurprisingly, the first principle component captures variation in count depth between cells, and is thus only marginally informative
The plot does not show the expected clustering of the data in two dimensions

t-SNE:

Shows several distinct clusters with clear subcluster structure
Connections between clusters are difficult to interpret visually

UMAP:

Data points are spread out on the plot showing several clusters
Connections between clusters can be readily identified

Diffusion Maps:

Shows connections between regions of higher density
Very clear trajectories are suggested, but clusters are less clear
Each diffusion component extracts heterogeneity in a different part of the data

Graph:

Shows a central cluster and several outer clusters
Shows clear connections from the central cluster (likely stem cells) to outer clusters
The strengths and weaknesses of the visualizations can readily be identified in the above plots. While t-SNE exaggerates differences, diffusion maps exaggerate transitions. Overall UMAP and force-directed graph drawings show the best compromise of the two aspects, however UMAP is much faster to compute (8s vs 114s here). UMAP has furthermore been shown to more accurately display the structure in the data.

In [ ]:
adata.obs.head()

## Cell cycle scoring

TBD?

## 3.3. Clustering

In [ ]:
sc.tl.leiden(adata, resolution=0.3, key_added='leiden_r03')
sc.tl.leiden(adata, resolution=0.5, key_added='leiden_r05')
sc.tl.leiden(adata, resolution=0.8, key_added='leiden_r08')
sc.tl.leiden(adata, resolution=1, key_added='leiden_r1')
sc.tl.leiden(adata, resolution=1.5, key_added='leiden_r1.5')

In [ ]:
sc.tl.louvain(adata, resolution=0.3, key_added='louvain_r03')
sc.tl.louvain(adata, resolution=0.5, key_added='louvain_r05')
sc.tl.louvain(adata, resolution=0.8, key_added='louvain_r08')
sc.tl.louvain(adata, resolution=1.0, key_added='louvain_r1')
sc.tl.louvain(adata, resolution=1.5, key_added='louvain1.5')

In [ ]:
sc.pl.umap(adata, color=['leiden_r03','leiden_r05','leiden_r05','leiden_r08','leiden_r1','leiden_r1.5'], legend_loc='on data',save= figdirectory + tissue +'_leiden.png')

In [ ]:
sc.pl.umap(adata, color=['louvain_r03','louvain_r05','louvain_r05','louvain_r08','louvain_r1','louvain1.5'], legend_loc='on data',save= figdirectory + tissue +'_louvain.png')

## 3.4.  Find marker genes per cluster

In [ ]:
sc.settings.verbosity = 2

In [ ]:
adata.obs['cell_group'] = adata.obs['ConditionName'].astype(str) + '_' + adata.obs['leiden_r03'].astype(str)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden_r03', use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False, save = figdirectory + tissue +'_merge_rankgenes.png')

In [ ]:
#### only upregulated marker genes are subsetted here
marker_genes = pd.Series()
for i in adata.obs.leiden_r03.cat.categories:
    marker_genes = marker_genes.append(sc.get.rank_genes_groups_df(adata, group=i, pval_cutoff=1e-4)['names'][:10])
marker_genes = marker_genes.unique()

In [ ]:
sc.pl.matrixplot(adata, var_names=marker_genes, swap_axes= "True", groupby='leiden_r03', standard_scale='var', save = figdirectory + tissue +'_merge_heatmap.png')

In [ ]:
adata.write(clusters_file)